In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import pandas as pd

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


Dataset (Same)

In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [7]:
dataset = datasets.MNIST(root="data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root="data", train=False, download=True, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=64)
test_loader  = DataLoader(test_dataset, batch_size=64)

CNN with 18 layers 

In [8]:
class CNN18(nn.Module):
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1), nn.ReLU(),
        )

        self.fc = nn.Linear(32, 10)

    def forward(self, x):
        x = self.features(x)
        x = x.mean([2, 3])  # Global Average Pooling
        return self.fc(x)

training and eval functions

In [9]:
def train_epoch(model, loader, optimizer, loss_fn):
    model.train()
    correct, total, loss_sum = 0, 0, 0

    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
        correct += (out.argmax(1) == y).sum().item()
        total += y.size(0)

    return loss_sum / len(loader), correct / total


def evaluate(model, loader, loss_fn):
    model.eval()
    correct, total, loss_sum = 0, 0, 0

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = loss_fn(out, y)

            loss_sum += loss.item()
            correct += (out.argmax(1) == y).sum().item()
            total += y.size(0)

    return loss_sum / len(loader), correct / total

loss_fn = nn.CrossEntropyLoss()

cnn-18 training

In [10]:
cnn = CNN18().to(device)
optimizer_cnn = optim.Adam(cnn.parameters(), lr=0.001)

for epoch in range(5):
    train_epoch(cnn, train_loader, optimizer_cnn, loss_fn)

cnn_test_loss, cnn_test_acc = evaluate(cnn, test_loader, loss_fn)

Resnet Frozen

In [11]:
resnet_frozen = models.resnet18(pretrained=True)
resnet_frozen.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
resnet_frozen.fc = nn.Linear(resnet_frozen.fc.in_features, 10)

for param in resnet_frozen.parameters():
    param.requires_grad = False
for param in resnet_frozen.fc.parameters():
    param.requires_grad = True

resnet_frozen = resnet_frozen.to(device)
optimizer_frozen = optim.Adam(resnet_frozen.fc.parameters(), lr=0.001)

for epoch in range(3):
    train_epoch(resnet_frozen, train_loader, optimizer_frozen, loss_fn)

frozen_test_loss, frozen_test_acc = evaluate(resnet_frozen, test_loader, loss_fn)

c:\Users\HARSH\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\HARSH\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Resnet finetuned

In [12]:
resnet_ft = models.resnet18(pretrained=True)
resnet_ft.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
resnet_ft.fc = nn.Linear(resnet_ft.fc.in_features, 10)

for name, param in resnet_ft.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

resnet_ft = resnet_ft.to(device)
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet_ft.parameters()), lr=0.0005)

for epoch in range(5):
    train_epoch(resnet_ft, train_loader, optimizer_ft, loss_fn)

ft_test_loss, ft_test_acc = evaluate(resnet_ft, test_loader, loss_fn)

Comparision

In [13]:
results = pd.DataFrame({
    "Model": [
        "CNN-18 (Scratch)",
        "ResNet-18 (Pretrained Frozen)",
        "ResNet-18 (Pretrained Fine-Tuned)"
    ],
    "Test Accuracy": [
        cnn_test_acc,
        frozen_test_acc,
        ft_test_acc
    ],
    "Test Loss": [
        cnn_test_loss,
        frozen_test_loss,
        ft_test_loss
    ]
})

print("\nFINAL COMPARISON TABLE")
print(results)


FINAL COMPARISON TABLE
                               Model  Test Accuracy  Test Loss
0                   CNN-18 (Scratch)         0.9770   0.083342
1      ResNet-18 (Pretrained Frozen)         0.6166   1.159819
2  ResNet-18 (Pretrained Fine-Tuned)         0.9110   0.285521
